In [3]:
import requests
import json
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import json
import csv
import scipy.io as sio
import os

In [1]:
import requests
import pandas as pd
import json

# Base URL components
url_base = 'https://opendata.sz.gov.cn/api/29200_00403589/1/service.xhtml?page='
app_key = 'YOUR_APP_KEY'  # Replace with your actual app key, avoid hard-coding sensitive information

# Initialize an empty DataFrame to append data for each page
df_append = pd.DataFrame()

# Loop over pages
for page in range(2):
    # Print progress every 50 pages
    if page % 50 == 0:
        print(f"Processing page: {page}")
    
    # Construct the request URL
    url = f"{url_base}{page}&rows=10000&appKey={app_key}"
    
    # Fetch and parse JSON data
    response = requests.get(url)
    json_data = response.text
    dic_raw = json.loads(json_data)
    
    # Check and display the data type and data length for debugging
    print(type(dic_raw))
    if 'data' in dic_raw:
        print(f"Data length: {len(dic_raw['data'])}")
    else:
        print("No data found in the response")
        continue  # Skip to the next iteration if no data is found

    # Create DataFrame for the current page data
    df = pd.DataFrame()

    # Populate DataFrame with relevant fields
    for i, data_entry in enumerate(dic_raw['data']):
        try:
            df.loc[i, 'GOLEN'] = data_entry.get('GOLEN')
            df.loc[i, 'SPEED'] = data_entry.get('SPEED')
            df.loc[i, 'TIME1'] = data_entry.get('TIME1')
            df.loc[i, 'PERIOD'] = data_entry.get('PERIOD')
            df.loc[i, 'EXPONENT'] = data_entry.get('EXPONENT')
            df.loc[i, 'BLOCKID'] = data_entry.get('BLOCKID')
            df.loc[i, 'GOTIME'] = data_entry.get('GOTIME')
        except KeyError as e:
            print(f"Missing field {e} on page {page}, entry {i}")
            continue  # Skip this entry if any field is missing
    
    # Append page data to cumulative DataFrame and save incrementally
    df_append = df_append.append(df, ignore_index=True)
    df_append.to_csv('test.csv', mode='a', header=False, index=False)
    
    # Clear df_append for the next batch if necessary (to save memory)
    df_append.drop(df_append.index, inplace=True)